In [1]:
%load_ext cudf.pandas
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
%load_ext cuml.accel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

cuML: Installed accelerator for sklearn.
cuML: Successfully initialized accelerator.


In [4]:
%%time
#read in the datasets
embeddings_df = pd.read_parquet('embeddings2025/embeddings.parquet')
captions_df   = pd.read_parquet('embeddings2025/captions.parquet')

#took 6.9s dataset is 2.68G

CPU times: user 2.4 s, sys: 923 ms, total: 3.32 s
Wall time: 1.59 s


In [5]:
# grab the embedding data 
X_embeddings=embeddings_df.values

# check size of embeddings
print(X_embeddings.shape)
# what's the data type?
print(X_embeddings.dtype)

(652671, 768)
float32


In [6]:
%%time

## KMeans on all the data
# 1. Instantiate the model
kmeans = KMeans(n_clusters=10, random_state=0, n_init='auto')

# 2. Fit and predict cluster labels
kmeans_labels = kmeans.fit_predict(X_embeddings)  

# 3. Inspect results
kmeans_centroids = kmeans.cluster_centers_
print("Labels:", kmeans_labels)

# 10  clusters took 48.7s on Brev: data is (652671 x 768d)
# 300 clusters took 21minutes 9s on Brev. same data

Labels: [8 1 4 ... 2 3 0]
CPU times: user 1.08 s, sys: 121 ms, total: 1.2 s
Wall time: 1.27 s


In [7]:
%%time

from sklearn.manifold import TSNE

# 1. Instantiate sklearn t-SNE
tsne = TSNE(
    n_components=2,          # target dimensionality
    perplexity=30,           # neighbors considered in high-D space (5-50) default 30
    learning_rate=200,       # typical: 10–1000 default 200
    n_iter=1000,             # optimization steps
    method='barnes_hut',     # fast approximation algorithm
    random_state=42,         # for reproducible results
    verbose=1                # show progress
)

# 2. Fit and transform with sklearn (CPU-based)
X_tsne_sklearn = tsne.fit_transform(X_embeddings)  # returns a NumPy array



CPU times: user 2min, sys: 6.83 s, total: 2min 7s
Wall time: 2min 7s
